In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
DATADIR = 'C:/Users/user/Documents/1_aa/ML project/Sign_Language_digits/Sign_Language_digits'
CATEGORIES = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


img_size=50

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_arr = cv2.resize(img_arr, (img_size,img_size))
                training_data.append([new_arr, class_num])
            except:
                pass
create_training_data()

In [ ]:
random.shuffle(training_data)
X = []
y = []
for feature, label in training_data:
    X.append(feature)
    y.append(label)
X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)
X = X/255.0

In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, train_size=0.7)
X_train, X_valid = X_train_val[:1200], X_train_val[1200:]
y_train, y_valid = y_train_val[:1200], y_train_val[1200:]
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)
y_valid = tf.keras.utils.to_categorical(y_valid, 10)

In [ ]:
np.random.seed(45)
tf.set_random_seed(45)
# plt.imshow((training_data[50][0]))

In [ ]:
# X_train.shape
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(img_size, img_size, 1)))
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
# model.add(tf.keras.layers.Dropout(rate=0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation='relu',  kernel_regularizer=tf.keras.regularizers.l2(l=0.05)))
model.add(tf.keras.layers.Dropout(rate=0.4))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(loss = tf.keras.losses.categorical_crossentropy,
             optimizer= tf.keras.optimizers.Adagrad(learning_rate=0.04),
             metrics = ['accuracy'])
model_history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid))

In [ ]:
pd.DataFrame(model_history.history).plot()
plt.grid()
plt.show()

In [ ]:
model.save('mytrain.model')

In [ ]:
model.evaluate(X_test, y_test)